<a href="https://colab.research.google.com/gist/ernestocl/0983861103570007179cffb96b83fafd/cnn-cifar10-tf11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.backend_config import tf
from tensorflow import keras as ks
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import os
from sklearn.preprocessing import LabelEncoder

from tensorflow.keras.applications import vgg16
from tensorflow.keras.models import Model
from keras.models import Sequential, load_model
from tensorflow.keras.optimizers import *

from keras.datasets import cifar10
from keras.utils.np_utils import to_categorical

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model

## Parameters
num_classes = 10
num_epochs = 200
num_predictions = 10

## Hyper-parameters
batch_size = 32



In [ ]:
# Aquí iria el codigo para conectarse a Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [ ]:
ANCHO_IMAGEN = 32
ALTURA_IMAGEN = 32
IMAGE_SIZE = (ANCHO_IMAGEN, ALTURA_IMAGEN)
CANALES_IMAGENES = 3


ruta_archivos = '/content/drive/MyDrive/MASTER EN DATA SCIENCE/Módulo 6 - Deep Learning/Módulo 6.1 - Deep Learning/6.3 Data Augmentation y Transfer Learning/Técnicas Avanzadas (perros y gatos)/'

In [ ]:
cifar10 = ks.datasets.cifar10

(x_train_img, y_train_img), (x_test_img, y_test_img) = cifar10.load_data()

x_train_scaled = x_train_img / 255.
x_test_scaled = x_test_img / 255.



In [ ]:
print("Etiquetas de train:", y_train_img)


# Convertir los datos en arrays de una dimension (vectores) es una buena mejora en el dataset
y_train_img = y_train_img.ravel()
y_test_img = y_test_img.ravel()


print("Etiquetas de train (en vector):", y_train_img)


In [ ]:
cifar10_labels = [
'airplane', # id 0
'automobile',
'bird',
'cat',
'deer',
'dog',
'frog',
'horse',
'ship',
'truck'
]

print('Number of labels: %s' % len(cifar10_labels))

In [ ]:
# Pintemos una muestra de las las imagenes del dataset MNIST

print('Train: X=%s, y=%s' % (x_train_img.shape, y_train_img.shape))
print('Test: X=%s, y=%s' % (x_test_img.shape, y_test_img.shape))

for i in range(9):

    plt.subplot(330 + 1 + i)
    plt.imshow(x_train_img[i], cmap=plt.get_cmap('gray'))
    plt.title(cifar10_labels[y_train_img[i]])

plt.subplots_adjust(hspace = 1)
plt.show()

In [ ]:
y_train_img

In [ ]:
#converting the output labels to one hot encoding
#y_train_img = to_categorical(y_train_img, len(cifar10_labels))
#y_test_img = to_categorical(y_test_img, len(cifar10_labels))

x_val_scaled = x_train_scaled[-10000:]
y_val_img = y_train_img[-10000:]

x_train_scaled = x_train_scaled[:-10000]
y_train_img = y_train_img[:-10000]

In [ ]:
y_train_img

In [ ]:
# Validamos el resultado
print('Train: X=%s, y=%s' % (x_train_scaled.shape, y_train_img.shape))
print('Test: X=%s, y=%s' % (x_test_scaled.shape, y_test_img.shape))
print('Validation: X=%s, y=%s' % (x_val_scaled.shape, y_val_img.shape))

In [ ]:
# creating data aguementaion object with required arguments
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.2,1.0],
    zoom_range=[0.5,1.0],
    featurewise_center=True,
    featurewise_std_normalization=True)

In [ ]:
# montamos la red neuronal
vgg16_model = vgg16.VGG16(include_top=False, weights='imagenet', input_shape=(ANCHO_IMAGEN, ALTURA_IMAGEN, CANALES_IMAGENES))

vgg16_model.summary()

In [ ]:
output = vgg16_model.layers[-1].output
new_output_layer = ks.layers.Flatten()(output)
vgg16_new_model = Model(vgg16_model.input, new_output_layer)

vgg16_new_model.summary()

In [ ]:
# Veamos como va lo de "congelar" capas de entrenamiento
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in vgg16_new_model.layers]
pd.DataFrame(layers, columns=['Layer Type','Layer Name', 'Layer Trainable'])

In [ ]:
trainable = False

for layer in vgg16_new_model.layers:
  if layer.name == "block4_conv1":
    trainable = True
  layer.trainable = trainable

pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in vgg16_new_model.layers]
pd.DataFrame(layers, columns=['Layer Type','Layer Name', 'Layer Trainable'])

In [ ]:
vgg16_new_model.summary()

In [ ]:
# Definamos el output de salida de la transformacion VGG16
output_from_vgg16 = vgg16_new_model.output_shape[1]
print(output_from_vgg16) #TODO

print(vgg16_new_model.output_shape)

In [ ]:
model_with_vgg16 = ks.Sequential()

model_with_vgg16.add(vgg16_new_model)

#model_with_vgg16.add(ks.layers.InputLayer(input_shape=(output_from_vgg16)))
model_with_vgg16.add(ks.layers.Dense(512, activation='relu', input_shape=(output_from_vgg16,)))
model_with_vgg16.add(ks.layers.Dropout(0.3))
model_with_vgg16.add(ks.layers.Dense(512, activation='relu'))
model_with_vgg16.add(ks.layers.Dropout(0.3))
model_with_vgg16.add(ks.layers.Dense(10, activation='softmax'))

model_with_vgg16.summary()

In [ ]:
tf.keras.optimizers.RMSprop(
    learning_rate=0.001,
    rho=0.9,
    momentum=0.0,
    decay=1e-6,
    epsilon=1e-07,
    centered=False,
    name="RMSprop")

In [ ]:
 # optimizer
opt = tf.keras.optimizers.RMSprop(lr=1e-4, decay=1e-6)

In [ ]:
model_with_vgg16.compile(loss='categorical_crossentropy',
                      optimizer=opt,
                      metrics=['accuracy'])

In [ ]:
callback = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
le = LabelEncoder()
le.fit(y_train_img)
y_train_encoded = le.transform(y_train_img)
y_val_encoded = le.transform(y_val_img)
y_test_encoded = le.transform(y_test_img)

In [ ]:
print(np.array(y_train_img))
print(np.array(y_train_encoded))

y_train_encoded

In [ ]:
# fitting training data to out datagen object
datagen.fit(x_train_scaled)

In [ ]:
history = model_with_vgg16.fit(datagen.flow(x_train_scaled, y_train_encoded, batch_size=batch_size),
                            steps_per_epoch=x_train_scaled.shape[0] // batch_size, epochs=num_epochs,
                            validation_data=(x_test_scaled, y_test_encoded))

In [ ]:
evaluation = history.evaluate_generator(datagen.flow(x_test_scaled, y_test_encoded, batch_size=batch_size), steps=x_test_scaled.shape[0] // batch_size)

print('Model accuracy = %.2f' % evaluation[1])

In [ ]:
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
plt.show()

plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')
plt.show()